Détection et sélection du catalog

In [0]:
catalogs = [row.catalog for row in spark.sql("SHOW CATALOGS").collect()]
unity_catalogs = [c for c in catalogs if c != "hive_metastore"]

if len(unity_catalogs) == 1:
    default_catalog = unity_catalogs[0]
else:
    # Choisir celui qui commence par 'dbw_' si plusieurs catalogs sont présents
    default_catalog = next((c for c in unity_catalogs if c.startswith("dbw_")), "hive_metastore")

# Création d'un widget pour sélectionner dynamiquement le catalog
dbutils.widgets.text("my_catalog", default_catalog, "Catalog détecté")
catalog = dbutils.widgets.get("my_catalog")


Définition des noms de schémas utilisés pour le projet

In [0]:
bronze_schema = "bronze"
silver_schema = "silver"
gold_schema = "gold"
logs_schema = "logs"


Fonction pour créer un schéma si inexistant dans le catalog

In [0]:
def create_schema_if_not_exists(schema_name):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_name}")
    print(f"Schéma {catalog}.{schema_name} prêt.")


Création des schémas Bronze, Silver, Gold et Logs

In [0]:
create_schema_if_not_exists(bronze_schema)
create_schema_if_not_exists(silver_schema)
create_schema_if_not_exists(gold_schema)
create_schema_if_not_exists(logs_schema)


Fonction générique pour créer une table de log si elle n'existe pas

In [0]:
def create_log_table_if_needed(table_name, schema):
    full_table_name = f"{catalog}.{logs_schema}.{table_name}"
    if not spark._jsparkSession.catalog().tableExists(full_table_name):
        empty_df = spark.createDataFrame([], schema)
        empty_df.write.format("delta") \
            .mode("overwrite") \
            .saveAsTable(full_table_name)
        print(f"Table {full_table_name} créée.")
    else:
        print(f"Table {full_table_name} déjà existante.")


Schémas de logs utilisés pour chaque couche

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

bronze_log_schema = StructType([
    StructField("table_name", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("status", StringType(), False),
    StructField("message", StringType(), True)
])

silver_log_schema = StructType([
    StructField("table_name", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("status", StringType(), False),
    StructField("rows_inserted", IntegerType(), True),
    StructField("message", StringType(), True)
])

gold_log_schema = StructType([
    StructField("table_name", StringType(), False),
    StructField("timestamp", TimestampType(), False),
    StructField("status", StringType(), False),
    StructField("rows_inserted", IntegerType(), True),
    StructField("message", StringType(), True)
])


Création des tables de log pour Bronze, Silver et Gold

In [0]:
create_log_table_if_needed("bronze_processing_log", bronze_log_schema)
create_log_table_if_needed("silver_processing_log", silver_log_schema)
create_log_table_if_needed("gold_processing_log", gold_log_schema)
